In [1]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from IPython.core.display import display, HTML

def scrape_table(table):
    df = pd.read_html(str(table))
    df = pd.DataFrame(df[0])
    return Table.from_df(df)

/var/folders/70/3whxqt_j51b6kx783xwzklbw0000gn/T/ipykernel_44189/1145054876.py:4: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


#1. Scrape data#

In [18]:
egg_price_url = "https://www.globalproductprices.com/rankings/egg_prices/"
tax_rate_url = "https://tradingeconomics.com/country-list/sales-tax-rate"
currency_url = "https://taxsummaries.pwc.com/glossary/currency-codes"

In [4]:
#Scrape data from egg_price_url
egg_price_request = requests.get(egg_price_url)
egg_soup = BeautifulSoup(egg_price_request.text, 'html.parser')

raw_price_table = egg_soup.find('table')
egg_price_table = scrape_table(raw_price_table)
egg_price_table

Countries,"Eggs prices, dozen",Rank
New Zealand,6.77,1
Switzerland,5.89,2
Norway,5.43,3
Uruguay,5.39,4
Australia,5.07,5
Bulgaria,4.86,6
South Korea,4.54,7
Sweden,4.39,8
Germany,4.22,9
Belgium,4.11,10


In [16]:
#Scrape data from tax_rate_url
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
tax_rate_request = requests.get(tax_rate_url, headers=headers)
tax_soup = BeautifulSoup(tax_rate_request.text, 'html.parser')

raw_tax_table = tax_soup.find('table')
tax_price_table = scrape_table(raw_tax_table)
tax_price_table

Country,Last,Previous,Reference,Unit
Afghanistan,10,10,Dec/24,%
Albania,20,20,Dec/24,%
Algeria,19,19,Dec/24,%
Andorra,4.5,4.5,Dec/24,%
Angola,14,14,Dec/23,%
Antigua And Barbuda,15,15,Dec/24,%
Argentina,21,21,Dec/24,%
Armenia,20,20,Dec/24,%
Aruba,4,4,Dec/24,%
Australia,10,10,Dec/24,%


In [24]:
#Collect currency data
currency_request = requests.get(currency_url, headers=headers)
currency_soup = BeautifulSoup(currency_request.text, 'html.parser')

raw_currency_table = currency_soup.find('table')
currency_table = pd.read_html(str(raw_currency_table))[0]
currency_table

,0,1,2
0,Territory,Currency code (ISO 4217),Currency name
1,Albania,ALL,lek
2,Algeria,DZD,dinar
3,Angola,AOA,kwanza
4,Argentina,ARS,peso
...,...,...,...
150,"Uzbekistan, Republic of",UZS,soum
151,Venezuela,VEF,bolivar
152,Vietnam,VND,dong
153,Zambia,ZMW,kwacha
